In [32]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pickle
from tflearning.data.creator import create_datasetgenerator, DataConfig
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
flowers_path = Path('/system/user/beck/pwbeck/projects/regularization/tflearning/outputs_scripts/prediction_depth/PD-flowers102-resnet18--230317_153444/prediction_depth_results.p')

In [34]:
def load_dict(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [35]:
pd_flowers = load_dict(flowers_path)
pd_flowers['train'].keys()

dict_keys(['layer_accs', 'pred_depths', 'layer_preds', 'labels', 'num_classes', 'entropies'])

In [36]:
len(pd_flowers['train']['entropies'])

1020

In [37]:
dcfg = DataConfig(name='flowers102', kwargs={'data_root_path': '/system/user/beck/pwbeck/data', 'n_px': 224})
dsgen_flowers = create_datasetgenerator(data_cfg=dcfg)
dsgen_flowers.generate_dataset()

/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/torchvision/transforms/transforms.py:891: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [38]:
dsgen_flowers.train_split

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /system/user/beck/pwbeck/data
    split=train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic)
               <function _convert_to_rgb at 0x7f58f4945f30>
               ToTensor()
               Normalize(mean=[0.4329557418823242, 0.38192424178123474, 0.2964075207710266], std=[0.2588456869125366, 0.20939205586910248, 0.22115884721279144])
           )

In [39]:
entr = pd_flowers['train']['entropies']

In [41]:
np.arange(len(entr))

array([   0,    1,    2, ..., 1017, 1018, 1019])

In [50]:
sorted_idxes = entr.argsort() # sorts ascending
entr[sorted_idxes[0]], entr[sorted_idxes[500]]

(2.220446049250313e-16, 1.5229550675313184)

In [44]:
sample_idx_entr = np.stack([np.arange(len(entr)), entr], axis=1)

In [ ]:
def get_sample_idxes_by_metric(sample_idx_metric : np.ndarray, fraction: float, best: bool = True) -> np.ndarray:
    """Returns the idxes of the `fraction` of samples with the best or worst metric value."""
    assert sample_idx_metric.ndim == 1, f"sample_idx_metric.ndim must be 1, but is {sample_idx_metric.ndim}"
    sorted_idxes = sample_idx_metric.argsort() # sorts ascending
    if best:
        sorted_idxes = sorted_idxes[::-1] # reverse order, so that best samples are first
    n_samples = int(len(sorted_idxes) * fraction)
    return sorted_idxes[:n_samples]

In [53]:
x = np.arange(10)
np.random.shuffle(x)
x

array([3, 5, 1, 2, 4, 6, 7, 0, 8, 9])

In [56]:
idx_sort = x.argsort()

In [58]:
x[idx_sort][::-1]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])